** This script generates a summary of the top categories. **
An Excel sheet would be generated for products in various categories that hve the highest sales


In [2]:
# Importing libaries
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter

In [3]:
# Storing the filepath
FILE = "amazon.xlsx"
# read the Excel file
df = pd.read_excel(FILE)
df = df[["category", "actual_price"]]
df.head

<bound method NDFrame.head of                                                category actual_price
0     Computers&Accessories|Accessories&Peripherals|...       ₹1,099
1     Computers&Accessories|Accessories&Peripherals|...         ₹349
2     Computers&Accessories|Accessories&Peripherals|...       ₹1,899
3     Computers&Accessories|Accessories&Peripherals|...         ₹699
4     Computers&Accessories|Accessories&Peripherals|...         ₹399
...                                                 ...          ...
1460  Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...         ₹919
1461  Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...       ₹3,045
1462  Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...       ₹3,080
1463  Home&Kitchen|Heating,Cooling&AirQuality|Fans|E...       ₹1,890
1464  Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...       ₹3,690

[1465 rows x 2 columns]>

In [4]:
# Data cleaning
df.isna().any()
# check for duplicated values
df.duplicated()
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      1465 non-null   object
 1   actual_price  1465 non-null   object
dtypes: object(2)
memory usage: 23.0+ KB


In [5]:
df["actual_price"] = df["actual_price"].str.replace("₹", "")
df["actual_price"] = df["actual_price"].str.replace(",", "")
df["actual_price"] = df["actual_price"].astype("float64")
df

,category,actual_price
0,Computers&Accessories|Accessories&Peripherals|...,1099.0
1,Computers&Accessories|Accessories&Peripherals|...,349.0
2,Computers&Accessories|Accessories&Peripherals|...,1899.0
3,Computers&Accessories|Accessories&Peripherals|...,699.0
4,Computers&Accessories|Accessories&Peripherals|...,399.0
...,...,...
1460,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,919.0
1461,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,3045.0
1462,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...",3080.0
1463,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|E...",1890.0


In [6]:
# summary chart
top_categorical_sales = (
    df.groupby("category")["actual_price"].sum().sort_values(ascending=True)
)
# export to Excel
with pd.ExcelWriter("top_categorical_sales.xlsx") as writer:
    top_categorical_sales.to_excel(writer, sheet_name="Sales Categories")

**Formatting the excel file with openpyxl**


In [7]:
# loading the Excel file
workbook = load_workbook("top_categorical_sales.xlsx")

# the sheet
sheet = workbook["Sales Categories"]

In [8]:
# setting up the font
header_font = Font(bold=True, name="DejaVu Serif")
header_fil = PatternFill(start_color="C70039", end_color="C70039", fill_type="solid")

for cell in sheet[1]:
    cell.font = header_font
    cell.fill = header_fil

In [9]:
MIN_WIDTH = 12  
for column_cells in sheet.iter_cols():
    max_length = 0
    column_letter = get_column_letter(column_cells[0].column)
    
    for cell in column_cells:
        if cell.value:
            try:
                cell_length = len(str(cell.value))
                if cell_length > max_length:
                    max_length = cell_length
            except None:
                pass

    # Ensure a minimum width
    adjusted_width = max(max_length + 2, MIN_WIDTH)
    sheet.column_dimensions[column_letter].width = adjusted_width


In [11]:
# export Excel file
workbook.save("top_products_per_categories.xlsx")